In [1]:
using SpeedyWeather, TravellingSailorProblem, GLMakie, LinearAlgebra

In [5]:
ntrunc = 31 # Spectral grid truncation level
nlayers = 8 # Number of layers in the atmosphere
particle_layer = 8 # Layer in which the particles are released
npartg = 4 # Number of particles per grid cell

4

In [4]:
# create a simulation
spectral_grid = SpectralGrid(trunc=ntrunc, nlayers=nlayers, Grid=HEALPixGrid)

# Three particles per grid square
nparticles = npartg*spectral_grid.npoints

# Re-create grid
spectral_grid = SpectralGrid(trunc=ntrunc, nlayers=nlayers, nparticles=nparticles, Grid=HEALPixGrid)

SpectralGrid{Spectrum{...}, HEALPixGrid{...}}
├ Number format: Float32
├ Spectral:      T31 LowerTriangularMatrix
├ Grid:          47-ring HEALPixGrid, 1728 grid points
├ Resolution:    4.89°, 543km (at 6371km radius)
├ Particles:     6912
├ Vertical:      8-layer atmosphere, 2-layer land
└ Architecture:  CPU using Array

In [ ]:
particle_advection = ParticleAdvection2D(spectral_grid, layer=particle_layer)
model = PrimitiveWetModel(spectral_grid; particle_advection)
simulation = initialize!(model)

# add particle tracker
particle_tracker = ParticleTracker(spectral_grid)
add!(model, :particle_tracker => particle_tracker)

### Getting the lon lat of the grid points

In [5]:
londs, latds = RingGrids.get_londlatds(spectral_grid.grid)

([45.0, 135.0, 225.0, 315.0, 22.5, 67.5, 112.5, 157.5, 202.5, 247.5  …  112.5, 157.5, 202.5, 247.5, 292.5, 337.5, 45.0, 135.0, 225.0, 315.0], [86.10076357950555, 86.10076357950555, 86.10076357950555, 86.10076357950555, 82.19700324028634, 82.19700324028634, 82.19700324028634, 82.19700324028634, 82.19700324028634, 82.19700324028634  …  -82.19700324028634, -82.19700324028634, -82.19700324028634, -82.19700324028634, -82.19700324028634, -82.19700324028634, -86.10076357950555, -86.10076357950555, -86.10076357950555, -86.10076357950555])

### Particle locations to adjust

In [6]:
# adjust initial locations of particles
(; particles) = simulation.prognostic_variables
particles

6912-element Vector{Particle{Float32}}:
 Particle{Float32}(  active, 113.08˚E,  68.86˚N, σ = 0.24)
 Particle{Float32}(  active, 140.28˚E,  47.64˚N, σ = 0.83)
 Particle{Float32}(  active,   8.67˚E,   1.88˚N, σ = 0.54)
 Particle{Float32}(  active,  43.25˚E,  65.72˚N, σ = 0.37)
 Particle{Float32}(  active, 157.74˚E, -24.22˚N, σ = 0.05)
 Particle{Float32}(  active, 223.02˚E, -32.50˚N, σ = 0.61)
 Particle{Float32}(  active,  56.85˚E,  -2.32˚N, σ = 0.84)
 Particle{Float32}(  active, 273.46˚E,  77.31˚N, σ = 0.71)
 Particle{Float32}(  active,  64.27˚E,  42.16˚N, σ = 0.31)
 Particle{Float32}(  active, 142.59˚E, -46.13˚N, σ = 0.25)
 ⋮
 Particle{Float32}(  active,  37.97˚E, -10.23˚N, σ = 0.47)
 Particle{Float32}(  active, 155.62˚E, -16.48˚N, σ = 0.59)
 Particle{Float32}(  active, 293.05˚E, -53.84˚N, σ = 0.45)
 Particle{Float32}(  active, 294.84˚E,  57.85˚N, σ = 0.23)
 Particle{Float32}(  active,  19.35˚E,  -1.04˚N, σ = 0.31)
 Particle{Float32}(  active, 337.74˚E, -31.14˚N, σ = 0.36)
 Particle{Flo

### Set particles to grid points and perturb

In [7]:
Re = 6.371e6; # Average Earth radius in meters
dist_km = 10 # Perturbation to apply - distance in km - Hyperparameter to experiment with later down the line
del_lat = rad2deg((dist_km * 1000 / Re)) # Latitude perturbation in degrees

0.08993216059187306

In [8]:
# Unperturbed - do not need to track

# Perturbed to the East
cos_factor = cos.(deg2rad.(latds))
particles[1:npartg:end] .= [Particle(londs[i] + del_lat/cos_factor[i], latds[i]) for i in 1:spectral_grid.npoints]
particles[2:npartg:end] .= [Particle(londs[i] - del_lat/cos_factor[i], latds[i]) for i in 1:spectral_grid.npoints]

# Perturbed to the North
particles[3:npartg:end] .= [Particle(londs[i], latds[i] + del_lat) for i in 1:spectral_grid.npoints]
particles[4:npartg:end] .= [Particle(londs[i], latds[i] - del_lat) for i in 1:spectral_grid.npoints]

particles

6912-element Vector{Particle{Float32}}:
 Particle{Float32}(  active,  46.32˚E,  86.10˚N, σ = 0.00)
 Particle{Float32}(  active,  43.68˚E,  86.10˚N, σ = 0.00)
 Particle{Float32}(  active,  45.00˚E,  86.19˚N, σ = 0.00)
 Particle{Float32}(  active,  45.00˚E,  86.01˚N, σ = 0.00)
 Particle{Float32}(  active, 136.32˚E,  86.10˚N, σ = 0.00)
 Particle{Float32}(  active, 133.68˚E,  86.10˚N, σ = 0.00)
 Particle{Float32}(  active, 135.00˚E,  86.19˚N, σ = 0.00)
 Particle{Float32}(  active, 135.00˚E,  86.01˚N, σ = 0.00)
 Particle{Float32}(  active, 226.32˚E,  86.10˚N, σ = 0.00)
 Particle{Float32}(  active, 223.68˚E,  86.10˚N, σ = 0.00)
 ⋮
 Particle{Float32}(  active, 135.00˚E, -86.19˚N, σ = 0.00)
 Particle{Float32}(  active, 226.32˚E, -86.10˚N, σ = 0.00)
 Particle{Float32}(  active, 223.68˚E, -86.10˚N, σ = 0.00)
 Particle{Float32}(  active, 225.00˚E, -86.01˚N, σ = 0.00)
 Particle{Float32}(  active, 225.00˚E, -86.19˚N, σ = 0.00)
 Particle{Float32}(  active, 316.32˚E, -86.10˚N, σ = 0.00)
 Particle{Flo

In [9]:
# then run! simulation
ndays = 10
run!(simulation, period=Day(ndays))

┌ Info: ParticleTracker writes to particles.nc as output=false
└ @ SpeedyWeather /home/teaching/.julia/packages/SpeedyWeather/voHFE/src/output/particle_tracker.jl:58


In [ ]:
# and visualise
#globe(particle_tracker)

### Compute gradient matrix using central differences

In [10]:
dfac = Re / dist_km / 2

# Derivative of x w.r.t. X
xX = [deg2rad((particles[4i-3].lon - particles[4i-2].lon))*cos_factor[i]*dfac for i in 1:spectral_grid.npoints];

# Derivative of y w.r.t. X 
yX = [deg2rad((particles[4i-3].lat - particles[4i-2].lat))*dfac for i in 1:spectral_grid.npoints];

# Derivative of x w.r.t. Y
xY = [deg2rad((particles[4i-1].lon - particles[4i].lon))*cos_factor[i]*dfac for i in 1:spectral_grid.npoints];

# Derivative of y w.r.t. Y 
yY = [deg2rad((particles[4i-1].lat - particles[4i].lat))*dfac for i in 1:spectral_grid.npoints];

In [11]:
# Looping is not the most efficient method but is clearer
# May want to replace with more efficient method later
FTLE = Vector{Float64}(undef, spectral_grid.npoints) 
for k in 1:spectral_grid.npoints
    # Gradient tensor
    B = [xX[k] xY[k];
        yX[k] yY[k]];
    # Cauchy-Green tensor
    CG = B' * B;
    # Largest eigenvalue
    lmax = maximum(eigvals(CG))
    # FTLE - in units of days^-1
    FTLE[k] = log(lmax)/2/ndays 
end

In [12]:
FTLE

1728-element Vector{Float64}:
 1.08173365439897
 1.060865484475559
 1.1461887958241204
 1.159092278818384
 1.2283161633750512
 1.2216275747044731
 1.0357269537341227
 1.1486806728928498
 1.219111295221661
 1.0736773336627377
 ⋮
 1.1886896856508886
 1.0095748713014996
 1.1587788896055418
 1.1783955780137734
 0.6717541185107907
 1.0660195178017264
 1.042876353440993
 1.1621819378841236
 1.070070601800273

In [13]:
grid = spectral_grid.grid
FTLE_grid = Field(FTLE, grid);

In [14]:
heatmap(FTLE_grid)

In [15]:
globe(rand(grid))